Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your github nickname below:

In [4]:
NICKNAME = "DziyanaC"

---

# Часть 1. Гауссовы квадратуры

Метод Гауссовых квадратур является одним из методов численного интегрирования, который позволяет увеличить точность интегрирования, используя ортогональные полиномы и их корни в качестве узлов.

$$
\int_a^b \!  w(x)\, f(x)\, dx \approx \sum_{k=1}^n w_k \, f(x_k) \;,
$$

здесь $\omega$ - функция весов, которая определят базис ортогональных полиномов, чьи корни используются в качестве узлов интегрирования. 

Получаемая таким образом квадратурная схема *точно* инегрирует $f(x)$ если это полином степени $m\le2n-1$, где $n$ - степень полинома, корни которого используются в качестве узлов.

## Двухточечная квадратура Гаусса

Постройте квадратурную формулу Гаусса, интегрирующую точно многочлены степеней вплоть до третьей на интервале $[a, b]$. Заметим, что для этого достаточно построить _двухточечную_ квадратуру.

    hint:Легко найти точки и веса для отрезка [-1,1], после чего можно сделать аффинное преобразование..

Напишите функцию, которая возвращает веса, $w_1$ и $w_2$, и узлы, $x_1$ и $x_2$, двухточечной квадратурной формулы Гаусса с весовой функцией $w(x) = 1$, т.е., интегралы вида

$$
\int_a^b\! f(x)\, dx \approx w_1 f(x_1) + w_2 f(x_2)
$$

In [13]:
import scipy.special
import numpy as np

def gauss_2(a, b):
    r"""Return nodes and weights for a two-point Gauss quadrature on [a, b].
    
    Parameters
    ----------
    a, b : float
       Boundaries of the integration interval
       
    Returns
    -------
    x1, x2, w1, w2 : float
       Nodes and weights of the quadrature.
    """
    roots, weights = scipy.special.roots_legendre(2)
    w1 = (b - a) / 2 * weights[0]
    w2 = (b - a) / 2 * weights[1]
    x1 = (a + b) / 2 + (b - a) / 2 * roots[0]
    x2 = (a + b) / 2 + (b - a) / 2 * roots[1]
    return x1, x2, w1, w2

In [14]:
from numpy.testing import assert_allclose

x1, x2, w1, w2 = gauss_2(0, 1)

def f(x, n): 
    return x**n

for n in [0, 1, 2, 3]:
    assert_allclose(w1*f(x1, n=n) + w2*f(x2, n=n),
                    1./(n+1), atol=1e-14)

## Использование ортогональных полиномов

### Legendre

Теперь посчитаем следующий интеграл:

$$
\int_{-1}^1 (7x^3 - 8 x^2 - 3 x + 3) dx  \label{eq1}\tag{1}
$$

Здесь функция уже полиномиальна, поэтому возьмем весовую функцию $\omega(x) = 1$. Тогда вычисление интеграла сводится к сумме корней полинома, соответствующих весовой функции $\omega(x) = 1$. Например, здесь https://dlmf.nist.gov/18.3 вы можете увидеть, какой базис полиномов соответствует вашей весовой функции.

Заметим, что нашей соответствуют полиномы Лежандра. Теперь, зайдем в документацию https://docs.scipy.org/doc/scipy/reference/special.html#orthogonal-polynomials. Функция *roots_legendre* сразу же выдаст корни полиномов Лежандра, а также веса к их корням (для полиномов Лежандра все эти веса равны единице).

1. Посчитайте точное значение интеграла ([1](#mjx-eqn-eq1)) "руками".

In [7]:
I = lambda x: (7 / 4) * x ** 4 - (8 / 3) * x ** 3 - (3 / 2) * x ** 2 + 3 * x
a = -1
b = 1
A = I(b) - I(a)
print(A)

0.6666666666666674


2. Посчитайте значение интеграла ([1](#mjx-eqn-eq1)), используя обычный метод интегрирования на равномерной сетке (например, написанный вами в предыдущем дз). Оцените количество узлов, необходимых для достижения относительной точности $10^{-10}$.

In [8]:
def midpoint_rule(func, a, b, N):
    h = (b - a) / N
    s1 = 0.0
    for i in range(N):
        s1 += func(a + 0.5 * h + i * h)
    return s1 * h

def trapezoid_rule(func, a, b, delta):
    N=1
    h = (b - a) / N
    s2 = 0.0
    s3 = 0.5 * h * (func(a) + func(b))
    error = 1
    while error > abs(delta * s3):
        s2 = s3
        s3 = 0.5 * (s3 + midpoint_rule(F, a, b, N)) 
        N *= 2
        error = abs(s3 - s2)
    return s3, N + 1

F = lambda x: 7 * x ** 3 - 8 * x ** 2 - 3 * x + 3
int, points = trapezoid_rule(F, -1, 1, delta = 1e-10)
print('Результат вычисления интеграла методом интегрирования на равномерной сетке', int, 'при количестве узлов:', points)

Результат вычисления интеграла методом интегрирования на равномерной сетке 0.6666666666569454 при количестве узлов: 1048577


3. Посчитайте значение интеграла ([1](#mjx-eqn-eq1)), с помощью Гауссовой квадратуры (используя корни полинома Лежандра степени от 1 до 6 в качестве узлов) (чтобы получить корни и веса используйте scipy.special.roots_legendre)

In [9]:
for i in range(1, 7):
    roots, weights = scipy.special.roots_legendre(i)
    print('степень полинома Лежандра:', i, ' -- численный ответ:', (F(roots) * weights).sum())

степень полинома Лежандра: 1  -- численный ответ: 6.0
степень полинома Лежандра: 2  -- численный ответ: 0.6666666666666674
степень полинома Лежандра: 3  -- численный ответ: 0.6666666666666639
степень полинома Лежандра: 4  -- численный ответ: 0.6666666666666667
степень полинома Лежандра: 5  -- численный ответ: 0.6666666666666685
степень полинома Лежандра: 6  -- численный ответ: 0.6666666666666695


Сравните количество узлов, необходимых для достижения одинаковой точности в обоих чисенных методах.

In [10]:
def quad_legendre(F, delta):
    s = 10
    i = 2/3
    N = 0
    while abs(s - i) > i * delta:
        N += 1
        roots, weights = scipy.special.roots_legendre(n)
        s = (F(roots) * weights).sum()
    return s, N

for i in range(3, 14):
    N1 = quad_legendre(F, 10**(-i))[1]
    N2 = trapezoid_rule(F, -1, 1, 10**(-i))[1]
    print("Точность:", 10**(-i)," кол-во узлов методом Гаусса:", N1, "  кол-во узлов методом трапеции:", N2)

Точность: 0.001  кол-во узлов методом Гаусса: 2   кол-во узлов методом трапеции: 257
Точность: 0.0001  кол-во узлов методом Гаусса: 2   кол-во узлов методом трапеции: 1025
Точность: 1e-05  кол-во узлов методом Гаусса: 2   кол-во узлов методом трапеции: 4097
Точность: 1e-06  кол-во узлов методом Гаусса: 2   кол-во узлов методом трапеции: 8193
Точность: 1e-07  кол-во узлов методом Гаусса: 2   кол-во узлов методом трапеции: 32769
Точность: 1e-08  кол-во узлов методом Гаусса: 2   кол-во узлов методом трапеции: 131073
Точность: 1e-09  кол-во узлов методом Гаусса: 2   кол-во узлов методом трапеции: 262145
Точность: 1e-10  кол-во узлов методом Гаусса: 2   кол-во узлов методом трапеции: 1048577
Точность: 1e-11  кол-во узлов методом Гаусса: 2   кол-во узлов методом трапеции: 4194305
Точность: 1e-12  кол-во узлов методом Гаусса: 2   кол-во узлов методом трапеции: 8388609
Точность: 1e-13  кол-во узлов методом Гаусса: 2   кол-во узлов методом трапеции: 33554433



Вычислите значение того же интеграла, но на интервале от 0 до 10.

$$
\int_{0}^{10} (7x^3 - 8 x^2 - 3 x + 3) dx 
$$

Заметим, что больше нельзя использовать полиномы Лежанда, потому что они определены на интервале (-1, 1). Однако всегда можно произвести *аффинное преобразование* для того, чтобы перейти к желаемой системе координат.

(20% итоговой оценки)

In [27]:
a = 0
b = 10
roots, weights = scipy.special.roots_legendre(6)
x = (b - a) / 2 * roots + (a + b) / 2
s = (b - a) / 2 * (F(x)*weights).sum()
print('Результат численного интегрирования методом Гауссовых квадратур: ', s)

Результат численного интегрирования методом Гауссовых квадратур:  14713.333333333336


Сравните результат с точным значением интеграла.

Можно было произвести преобразование, изменив пределы интегрирования. В таком случае можно было бы воспользоваться полином Лежанда $$\int_{0}^{10} (7x^3 - 8 x^2 - 3 x + 3) dx =\{y = \frac{x}{10}\}= \int_{0}^{1} 10\cdot (7000y^3 - 800y^2 - 30y + 3) dy$$
Точное значение интеграла: $\frac{44140}{3}$.
Точное значение интергала очень хорошо совпадает с инткгралом, вычисленным интегрированием методом Гауссовых квадратур.

### 

Вычислите значение следующиего интеграла:
$$
\int_{0}^{2\pi} \frac{cos(x) dx }{\sqrt{4\pi^2 - x^2}}
$$
используя соответствующий базис ортогональных полиномов.
Найдите степень полинома (количество узлов), при которой точность превысит машинное $\epsilon$.


Преобразуем интеграл и произведем замену $y = \frac{x}{2\pi}$. Последнее равенство верно в силу четности функции.
$$
\int \limits_{0}^{2\pi} \frac{cos(x) dx }{\sqrt{4\pi^2 - x^2}} = \int \limits_{0}^{1} \frac{cos(2\pi y) dy }{\sqrt{1 - y^2}} = \frac{1}{2} \int \limits_{-1}^{1} \frac{cos(2\pi y) dy }{\sqrt{1 - y^2}}
$$
Будем использовать полином Чебышева первого рода.

In [15]:
F = lambda y: 0.5 * np.cos(2 * np.pi * y)
eps = 1e-16
real_s = 0.34601015881226444
s = 10000
N = 1
while abs(s - real_s) >= eps:
    roots, weights = scipy.special.roots_chebyt(N)
    s = (F(roots)*weights).sum()
    N += 1
print('Степень полинома Чебышева, при которой точность превысит машинное  𝜖 = 1e-16 равна', N)

Степень полинома Чебышева, при которой точность превысит машинное  𝜖 = 1e-16 равна 17


# Часть 2. Уравение Фредгольма

Существует два типа уравнений Фредгольма: 

1. Уравение Фредгольма первого рода:
$$
\int_a^b \! K(x, t)\, \color{blue}{f(t)} \,dt = g(x)
$$

2. Уравнение Фредгольма второго рода:
$$
\lambda \int_a^b\! K(x, t)\, \color{blue}{f(t)} \, dt + g(x) = \color{blue}{f(x)}
$$

Здесь функция $f(x)$ неизвестна, $K(x, t)$ - ядро уравнения, заданная функция $g(x)$ и $\lambda$ - рациональное число.
С точки зрения вычислений, уравение Фредгольма первого рода крайне неустойчиво. Далее мы рассмотрим только уравнение второго рода.

Основная идея - аппроксимировать интеграл некой квадратурной формулой

$$
\int_a^b \! \xi(t)\, dt \approx \sum_j^N w_j\, \xi(t_j)
$$

с подходящими весами $w_j$ и узлами $t_j$, $j=1, \dots, N$. Точность аппроксимации контролируется $N$.

Таким образом, уравнение Фредгольма приближается следующим образом:

$$
\lambda \sum_j w_j\, K(x, t_j)\, \color{blue}{f(t_j)} + g(x) = \color{blue}{f(x)}
$$

Отметим, что $x$ - непрерывная переменная, в то время, как $t$ - дискретная.

Рассматривая данное уравнение на тех же узлах сетки $x = t_k$, получаем

$$
\lambda \sum_j w_j \, K_{k, j}\, \color{blue}{f_j} + g_k = \color{blue}{f_k}
$$

где $f_j \equiv f(t_j)$, $g_j \equiv g(t_j)$ and $K_{k, j} \equiv K(t_k, t_j)$. Это ничто иное, как система линейных алгебраических уравнений на вектор $\color{blue}{f_j}$.

Ее решение выдает значение неизвестной функции $f(x)$ в дискретных точках $x=t_j$ для $j=1, \dots, N$.


Решите предложенное уравнение Фредгольма второго рода

$$
f(x) = \cos(x)  + \frac{1}{2\pi} \int_0^\pi \! \sin(x) \,t f(t) \, dt
$$

Здесь $a, b = 0, 1$, $\lambda = \dfrac{1}{2\pi}$, ядро $K(x, t) = \sin(x)\cdot t$ и правая часть - $g(x) = \cos(x)$.

Точное решение (Сборник задач по интегральным уравнениям, 2006):

$$
f(x) = \cos(x)- \frac{2}{\pi}\sin(x)
$$

Для нахождения интеграла преобразуйте заменой переменных в интеграле область интегрирования в [0,1] воспользуйтесь Гауссовой квадратурой с весовой функцией $w(x) = 1$ на $(0, 1)$. Посмотрите на http://dlmf.nist.gov/18.3, и найдите соответствующую функцию в *scipy.special* (обратите внимание на область интегрирования).

Напишите функцию, возвращающую значение f(t_j) по схеме выше, которая в качестве параметра принимает величину $N$--число узлов в квадратуре, для нахождение решения СЛАУ можно использовать `scipy.linalg.solve`.

Постройте график истинного решения и набор (t_j, f_j) для раличных $N$, используя вашу функцию.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

Очевидной проблемой предыдущего результата для $f(x)$ является то, что он возвращает решения на фиксированном наборе точек. Мы не контролируем их расположение.

Оказывается, интерполяционные методы (мы обсудим их на следующих занятиях) работают не очень хорошо, в сравнении с следующим подходом:

Используйте изначальный вид, чтобы вычислить значение функции на всём интервале:

$$
f(x) = g(x) + \lambda \sum_j^N w_j\, K(x, t_j)\, f(t_j)
$$

Сравните эту интерполяцию с точным решением для разлиных $N$.
(выбирайте наглядный масштаб)

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

# Часть 3. Интеграл от осциллирующей функции - EXTRA points task

Рассмотрим определенный интеграл

$$
I = \int_0^\infty\! \frac{\sin(x) \cos{(\cos{(x)})}}{x}\,dx
$$

Вычислите значение данного интеграла с относительной точностью $10^{-6}$. Для упрощения задачи можете воспользоваться функционалом `scipy.integrate.quad`.

Заметим, что "из коробки" интеграл вычислить не удается, и нужно что-то придумать.

In [ ]:
from math import sin, cos

from scipy.integrate import quad
quad(lambda x: sin(x) * cos(cos(x)) / x, 0, float('inf'))

Напишите функцию, которая возвращает значение данного интеграла. Только само значение интеграла, без оценки погрешности.
(оборачиваем интеграл в функцию только для удобства автопроверки).

In [ ]:
from scipy.integrate import quad

def integ():
    """Return a single float, the computed value of the integral."""
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
from numpy.testing import assert_allclose

# this is a test cell, keep it intact
